---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

def blight_model():
    pass
    


In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

# Load relevant data
df_train = pd.read_csv('train.csv', encoding = 'latin1')
df_test = pd.read_csv('test.csv')
df_address =  pd.read_csv('addresses.csv')
df_location = pd.read_csv('latlons.csv')

# Drop features which may result in leakage
leakage = ['payment_amount', 'payment_date', 'payment_status', 'collection_status', 'balance_due', 'compliance_detail' ] 
df_train.drop(leakage, axis=1, inplace=True)


# Create list of columns in the dataframe with dtype == string, since the classifier cannot evaluate them, we need to remove them. 
string_cols_to_drop_train = []
for col in range(0, len(df_train.dtypes)):
    if df_train.dtypes[col] == object:
        string_cols_to_drop_train.append(df_train.columns[col])

string_cols_to_drop_test = []
for col in range(0, len(df_test.dtypes)):
    if df_train.dtypes[col] == object:
        string_cols_to_drop_test.append(df_test.columns[col])
        
print('Cols to drop in train', string_cols_to_drop_train)
print('Cols to drop in test', string_cols_to_drop_test)

# Drop leakage cols and unnecessary cols
#df_train.drop(leakage, axis=1, inplace=True)
df_train.drop(string_cols_to_drop_train, axis=1, inplace=True)
df_test.drop(string_cols_to_drop_test, axis=1, inplace=True)

# Remove NaN values from compliance column
df_train = df_train[df_train['compliance'].notna()]

# Join the addresses and location frames on the common address index
df_address = df_address.set_index('address').join(df_location.set_index('address'), how='left')

# Join the above df to the training data on the common ticket_id index
df_train = df_train.set_index('ticket_id').join(df_address.set_index('ticket_id'))
df_test = df_test.set_index('ticket_id').join(df_address.set_index('ticket_id'))

# Drop remaining NaN values
df_train.dropna(axis=1, how='any', inplace=True)
df_test.dropna(axis=1, how='any', inplace=True)

# Create testing data
X_train = df_train.drop('compliance', axis=1)
y_train = df_train['compliance']
X_test = df_test

# Peform feature scaling
scaler = MinMaxScaler() # normalize features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)




c:\users\nate\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Cols to drop in train ['agency_name', 'inspector_name', 'violator_name', 'violation_street_name', 'mailing_address_str_name', 'city', 'state', 'zip_code', 'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date', 'violation_code', 'violation_description', 'disposition', 'grafitti_status']
Cols to drop in test ['agency_name', 'inspector_name', 'violator_name', 'violation_street_name', 'mailing_address_str_name', 'city', 'state', 'zip_code', 'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date', 'violation_code', 'violation_description', 'disposition', 'grafitti_status']


In [61]:
# Initialize the SVC and train the model
lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)

# Use the trained model to predict target values for test set
y_proba = lr.predict_proba(X_test_scaled)[:,1]

# Reload test and add your predicted target probabilities to return
df_test = pd.read_csv('test.csv')
df_test['compliance'] = y_proba
df_test.set_index('ticket_id', inplace=True)
df_test = df_test.iloc[:, -1]
df_test

ticket_id
284932    0.069278
285362    0.012626
285361    0.084881
285338    0.069287
285346    0.084903
            ...   
376496    0.012628
376497    0.012628
376499    0.084890
376500    0.084890
369851    0.109248
Name: compliance, Length: 61001, dtype: float64

In [38]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc

y_score_lr = lr.decision_function(X_test_scaled)
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_lr)
roc_auc_lr = auc(fpr_lr, tpr_lr)

plt.figure()
plt.xlim([-0.01, 1.00])
plt.ylim([-0.01, 1.01])
plt.plot(fpr_lr, tpr_lr, lw=3, label='LogRegr ROC curve (area = {:0.2f})'.format(roc_auc_lr))
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve (1-of-10 digits classifier)', fontsize=16)
plt.legend(loc='lower right', fontsize=13)
plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
plt.axes().set_aspect('equal')
plt.show()

NameError: name 'y_test' is not defined

In [40]:
df_test

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,compliance
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,0.069278
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN,0.012626
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,0.084881
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,0.069287
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,0.084903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60996,376496,"Buildings, Safety Engineering & Env Department","Pierson, Kevin",THE AIC GROUP,12032.0,SANTA ROSA,48204,P.O. BO,969,Southfield,...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN,0.012628
60997,376497,"Buildings, Safety Engineering & Env Department","Pierson, Kevin",THE AIC GROUP,12032.0,SANTA ROSA,48204,P.O. BO,969,Southfield,...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN,0.012628
60998,376499,Detroit Police Department,"BOWLES, TIFFANI","BARLOW, CHRISTOPHER D",11832.0,KILBOURNE,48213,11832,KILBOURNE,DETROIT,...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,0.084890
60999,376500,Detroit Police Department,"BOWLES, TIFFANI","WILLIAMS, JASON",11848.0,KILBOURNE,48213,4317,YORKSHIRE,DETROIT,...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,0.084890


In [3]:
import pandas as pd
import numpy as np

def blight_model():
    from sklearn.linear_model import LogisticRegression
    from sklearn.preprocessing import MinMaxScaler

    # Load relevant data
    df_train = pd.read_csv('train.csv', encoding = 'latin1')
    df_test = pd.read_csv('test.csv')
    df_address =  pd.read_csv('addresses.csv')
    df_location = pd.read_csv('latlons.csv')

    # Drop features which may result in leakage
    leakage = ['payment_amount', 'payment_date', 'payment_status', 'collection_status', 'balance_due', 'compliance_detail' ] 
    df_train.drop(leakage, axis=1, inplace=True)


    # Create list of columns in the dataframe with dtype == string, since the classifier cannot evaluate them, we need to remove them. 
    string_cols_to_drop_train = []
    for col in range(0, len(df_train.dtypes)):
        if df_train.dtypes[col] == object:
            string_cols_to_drop_train.append(df_train.columns[col])

    string_cols_to_drop_test = []
    for col in range(0, len(df_test.dtypes)):
        if df_train.dtypes[col] == object:
            string_cols_to_drop_test.append(df_test.columns[col])

    # Drop leakage cols and unnecessary cols
    #df_train.drop(leakage, axis=1, inplace=True)
    df_train.drop(string_cols_to_drop_train, axis=1, inplace=True)
    df_test.drop(string_cols_to_drop_test, axis=1, inplace=True)

    # Remove NaN values from compliance column
    df_train = df_train[df_train['compliance'].notna()]

    # Join the addresses and location frames on the common address index
    df_address = df_address.set_index('address').join(df_location.set_index('address'), how='left')

    # Join the above df to the training data on the common ticket_id index
    df_train = df_train.set_index('ticket_id').join(df_address.set_index('ticket_id'))
    df_test = df_test.set_index('ticket_id').join(df_address.set_index('ticket_id'))

    # Drop remaining NaN values
    df_train.dropna(axis=1, how='any', inplace=True)
    df_test.dropna(axis=1, how='any', inplace=True)

    # Create testing data
    X_train = df_train.drop('compliance', axis=1)
    y_train = df_train['compliance']
    X_test = df_test

    # Peform feature scaling
    scaler = MinMaxScaler() # normalize features
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Initialize the SVC and train the model
    lr = LogisticRegression()
    lr.fit(X_train_scaled, y_train)

    # Use the trained model to predict target values for test set
    y_proba = lr.predict_proba(X_test_scaled)[:,1]

    # Reload test and add your predicted target probabilities to return
    df_test = pd.read_csv('test.csv')
    df_test['compliance'] = y_proba
    df_test.set_index('ticket_id', inplace=True)
    df_test = df_test.iloc[:, -1]
    
    return df_test




In [4]:
blight_model()

<ipython-input-4-baf5c1d6201e>:1: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  blight_model()


ticket_id
284932    0.069278
285362    0.012626
285361    0.084881
285338    0.069287
285346    0.084903
            ...   
376496    0.012628
376497    0.012628
376499    0.084890
376500    0.084890
369851    0.109248
Name: compliance, Length: 61001, dtype: float64